In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from imutils import contours
import shutil

%matplotlib inline

TRAINING_PATH = 'training_data/'
BATCH_SIZE = 16
IMG_SHAPE = (64, 64)

In [2]:
from PIL import Image


class RandomShift(object):
    def __init__(self, shift):
        self.shift = shift
        
    @staticmethod
    def get_params(shift):
        """Get parameters for ``rotate`` for a random rotation.
        Returns:
            sequence: params to be passed to ``rotate`` for random rotation.
        """
        hshift, vshift = np.random.uniform(-shift, shift, size=2)

        return hshift, vshift 
    def __call__(self, img):
        hshift, vshift = self.get_params(self.shift)
        
        return img.transform(img.size, Image.AFFINE, (1,0,hshift,0,1,vshift), resample=Image.BICUBIC, fill=1)
    

In [3]:
# define basic image transforms for preprocessing
transform = transforms.Compose(
[
    transforms.ToPILImage(),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(0.2),
    RandomShift(0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.5,), std = (0.5, ))
])

class CrosswordDataset(torch.utils.data.Dataset):
    '''
    Custom Dataset object for the CDiscount competition
        Parameters:
            root_dir - directory including category folders with images

        Example:
        images/
            1000001859/
                26_0.jpg
                26_1.jpg
                ...
            1000004141/
                ...
            ...
    '''
    
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.categories = sorted(os.listdir(root_dir))
        self.cat2idx = dict(zip(self.categories, range(len(self.categories))))
        self.idx2cat = dict(zip(self.cat2idx.values(), self.cat2idx.keys()))
        self.files = []
        cat_mapping = {}
        for (dirpath, dirnames, filenames) in os.walk(self.root_dir):
            for f in filenames:
                if f.endswith('.png'):
                    o = {}
                    o['img_path'] = dirpath + '/' + f
                    o['category'] = self.cat2idx[dirpath[dirpath.find('/')+1:]]
                    cat_mapping[o['category']] = dirpath.split('/')[-1]
                    self.files.append(o)
        self.transform = transform
        print(f'Categorie mapping: {cat_mapping}')
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        img_path = self.files[idx]['img_path']
        category = self.files[idx]['category']
        image = cv2.imread(img_path)
        image = cv2.resize(image, IMG_SHAPE)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
            
        return {'image': image, 'category': category}


# create instance of data class and pytorch dataloader
dataSet = CrosswordDataset(TRAINING_PATH, transform=transform)
dataloader = torch.utils.data.DataLoader(dataSet, batch_size=BATCH_SIZE, shuffle=True)

Categorie mapping: {7: 'text', 0: 'down', 4: 'right', 6: 'right_down', 3: 'left_down', 8: 'up_right', 2: 'empty', 5: 'right_and_down', 1: 'down_right'}


In [4]:
class Net(nn.Module):
    # Pytorch CNN model class
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        self.conv3 = nn.Conv2d(16, 32, 5)
        self.conv4 = nn.Conv2d(32, 64, 5)
        
        
        self.dropout = nn.Dropout(0.3)
        
        self.fc1 = nn.Linear(64*11*11, 512)
        self.bnorm1 = nn.BatchNorm1d(512)
        
        self.fc2 = nn.Linear(512, 128)
        self.bnorm2 = nn.BatchNorm1d(128)
        
        self.fc3 = nn.Linear(128, 64)
        self.bnorm3 = nn.BatchNorm1d(64)
        
        self.fc4 = nn.Linear(64, 9)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        
        x = F.relu(self.conv3(x))
        x = self.pool(F.relu(self.conv4(x)))
        
        x = x.view(-1, 64*11*11)
        x = self.dropout(x)
        x = F.relu(self.bnorm1(self.fc1(x)))
        x = F.relu(self.bnorm2(self.fc2(x)))
        x = F.relu(self.bnorm3(self.fc3(x)))
        x = self.fc4(x)
        return x


net = Net()
print('######### Network created #########')
print('Architecture:\n', net)

### Train
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

print('Started Training!')
net.train()
for epoch in range(15):
    running_loss = 0.0
    examples = 0
    for i, data in enumerate(dataloader, 0):
        # Get the inputs
        inputs, labels = data['image'], data['category']
        
        # Wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.data
        examples += BATCH_SIZE
    print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / examples))

print('Finished Training!')

######### Network created #########
Architecture:
 Net(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=7744, out_features=512, bias=True)
  (bnorm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (bnorm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (bnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=64, out_features=9, bias=True)
)
Started Training!


/tmp/ipykernel_305739/3120566653.py:20: DeprecationWarning: AFFINE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transform.AFFINE instead.
  return img.transform(img.size, Image.AFFINE, (1,0,hshift,0,1,vshift), resample=Image.BICUBIC, fill=1)
/tmp/ipykernel_305739/3120566653.py:20: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  return img.transform(img.size, Image.AFFINE, (1,0,hshift,0,1,vshift), resample=Image.BICUBIC, fill=1)


[1,    60] loss: 0.068
[2,    60] loss: 0.035
[3,    60] loss: 0.027
[4,    60] loss: 0.022
[5,    60] loss: 0.021
[6,    60] loss: 0.016
[7,    60] loss: 0.016
[8,    60] loss: 0.015
[9,    60] loss: 0.014
[10,    60] loss: 0.013
[11,    60] loss: 0.014
[12,    60] loss: 0.011
[13,    60] loss: 0.012
[14,    60] loss: 0.016
[15,    60] loss: 0.011
Finished Training!


In [5]:
# save torch model for further prediction
torch.save(net.state_dict(), 'model.pt')